In [1]:
import pickle
import os
import re
import numpy as np
import spacy
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.preprocessing.sequence import pad_sequences
from datetime import datetime
from sklearn.preprocessing import normalize as scikit_normalize
from evaluation import plot_history
from evaluation import rmse_report
from sampling import UnderSampler3D
from fasttext_embedding import FastTextEmbeddingBag
from spacy.lang.en.stop_words import STOP_WORDS
from tqdm import tqdm

%matplotlib inline
%load_ext tensorboard

E0825 14:39:56.289077 140170260154176 ultratb.py:149] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kvassay/.virtualenvs/main/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-dd0197509285>", line 13, in <module>
    from sampling import UnderSampler3D
  File "/home/kvassay/data/z/test/nb/sampling.py", line 4, in <module>
    from imblearn.under_sampling import RandomUnderSampler
  File "/home/kvassay/.virtualenvs/main/lib/python3.6/site-packages/imblearn/__init__.py", line 34, in <module>
    from . import combine
  File "/home/kvassay/.virtualenvs/main/lib/python3.6/site-packages/imblearn/combine/__init__.py", line 5, in <module>
    from ._smote_enn import SMOTEENN
  File "/home/kvassay/.virtualenvs/main/lib/python3.6/site-packages/imblearn/combine/_smote_enn.py", line 10, in <module>
    from ..base import BaseSampler
  File "/home/kvassay/.virtualenvs/main/lib/python3.6/site-packages/imblearn/base.py", line 17, in <mo

KeyboardInterrupt: 

In [ ]:
DATASET='/home/kvassay/data/z/data/reviews_train_test_dev1_{}.pickle'
TYPE='tok'
TB_LOG_DIR='/home/kvassay/data/z/log/E3/scalars/'
VEC_DIM=30
DTYPE='float32'
FASTTEXT='/home/kvassay/data/z/models/fasttext/cbow_{}_e{}_w{}.bin'.format(VEC_DIM,20,5)
SEQ_PADDING=10
ALLOWED_SPECIAL=tuple(['?','!',':(', ':)', ':D',':-)',':-D',':\'(',':/',':-/','<3',':-P',':P'])

## Read data

In [ ]:
%%time
with open(DATASET.format(TYPE),'rb') as f:
    train,dev,_=pickle.load(f)

## Load models

In [ ]:
%%time
fasttext=FastTextEmbeddingBag(FASTTEXT)

## Preprocess text + extract features
- filter out EN stop-words (and, or, ...)
- filter out non-allowed special tokens (we want to keep smileys and !,?)

In [ ]:
rx_special= re.compile("|".join(re.escape(s) for s in ALLOWED_SPECIAL))

def word_filter(word):
    if word in STOP_WORDS:
        return False
    if not word.isalpha():
        if not rx_special.findall(word):
            return False
    else:
        if len(word)<3:
            return False
    return True

def preprocess_text(text):
    return [x.lower() for x in text if word_filter(x.lower())]

def preprocess_texts(dataset,text_keys=['summary','text']):
    for sample in tqdm(dataset):
        for key in text_keys:
            sample[key]=preprocess_text(sample[key])
    return dataset

In [ ]:
%%time
train=preprocess_texts(train)
dev=preprocess_texts(dev)

## Extract features
- transform texts to averages of their fastText vectors
- concatenate summary & text average vectors into single one

In [ ]:
def pad_vecs(vecs_mtx,length,dtype=DTYPE):
    return pad_sequences(vecs_mtx,
                         maxlen=length,
                         dtype=dtype,
                        padding='post',
                        truncating='post')

def extract_features(dataset, fasttext):
    default_vec=np.zeros(VEC_DIM,dtype=np.float32)
    vecs_all=[]
    for sample in tqdm(dataset):
        all_words=sample['summary']+sample['text']
        if all_words:
            vecs=fasttext.forward([x for x in all_words])
        else:
            vecs=np.array([default_vec])
        vecs=scikit_normalize(vecs)
        vecs=vecs.reshape(1,vecs.shape[0],vecs.shape[1])         
        vecs = pad_vecs(vecs, SEQ_PADDING)
        vecs_all.append(vecs)
    vecs_all=np.array(vecs_all)
    vecs_all=vecs_all.reshape(vecs_all.shape[0],vecs_all.shape[2],vecs_all.shape[3])
    return vecs_all

In [ ]:
%%time
X_train=extract_features(train,fasttext)
X_dev=extract_features(dev,fasttext)
y_train=np.array([x['score'] for x in train])
y_dev=np.array([x['score'] for x in dev])
print('Train samples shape: {}, Dev samples shape: {}'.format(X_train.shape,X_dev.shape))

#### Free up memory

In [ ]:
del train
del dev
del _ 

## Train
- shallow sequential NN

In [ ]:
K.backend.set_floatx(DTYPE)

In [ ]:
def get_tb_callback():
    suffix=datetime.now().strftime("%Y%m%d-%H%M%S")
    log_dir= os.path.join(TB_LOG_DIR,suffix)
    return K.callbacks.TensorBoard(log_dir=os.path.join(log_dir))

def train_model(batch_size,learning_rate, epochs):
    tensorboard_callback = get_tb_callback()
    model = K.models.Sequential([
        K.layers.Conv1D(64, 5, activation='relu',input_shape=(X_train.shape[1],X_train.shape[2])),
        Klayers.GlobalMaxPooling1D(),
        K.layers.Dense(10, activation='relu'),
        K.layers.Dense(1,activation='linear'),
    ])
    opt=K.optimizers.Adam(lr=learning_rate, decay=learning_rate/epochs)
    model.compile(optimizer=opt, loss='mean_squared_error')
    sampler=UnderSampler3D(X_train,y_train,batch_size=batch_size)
    model.fit_generator(sampler,
                        shuffle=False,
                        epochs=epochs,
                        validation_data=(X_dev,y_dev),
                        callbacks=[tensorboard_callback])
    return model

## Experiment

In [ ]:
def experiment(sampling_cls,learning_rate,epochs,batch_size,name):
    model=train_model(epochs=epochs,batch_size=batch_size,learning_rate=learning_rate)
    y_pred_dev=model.predict(X_dev)
    rmse_report(y_dev,y_pred_dev,title='{} - RMSE report'.format(name))
    plot_history(model,title='{} - Train/Dev MSE'.format(name))
    return model

In [ ]:
model=experiment(sampling_cls=UnderSampler,learning_rate=0.05, epochs=10,batch_size=512,name='CNN model')

In [ ]:
model.summary()